In [2]:
#Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import argparse
import json
import pprint
import requests
import sys
import urllib
import os 
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [3]:
#Assign database file to a variable
db_file = "../Resources/Los_Angeles/crimedata.db"

#Connect to database file
conn = sqlite3.connect(db_file)

In [162]:
#Query the la crime data table and put into a pandas dataframe
crime_df = pd.read_sql_query("select * from sherry_la_crime_clean_data;", conn)
crime_df.head()

,AreaID,ReportingDistrict,CrimeCode,VictimAge,PremiseCode
0,2,201,510,16.0,101.0
1,2,275,510,16.0,101.0
2,3,325,510,16.0,101.0
3,4,421,510,16.0,101.0
4,5,566,510,16.0,101.0


In [163]:
crime_df.count()

AreaID               174766
ReportingDistrict    174766
CrimeCode            174766
VictimAge            174766
PremiseCode          174766
dtype: int64

In [164]:
# Most victim age crimes are commited against
ages = crime_df.VictimAge.value_counts()
ages.head(11)

17.0    12516
16.0    12093
0.0     11503
26.0     4411
27.0     4323
25.0     4271
28.0     4179
29.0     4131
24.0     4063
30.0     3966
31.0     3960
Name: VictimAge, dtype: int64

In [165]:
area_ID = crime_df.AreaID.value_counts() 
area_ID.head(10)

3     26280
1     24183
6     20940
7     18542
2     18192
9     17306
5     16300
4     15293
8     15290
13      816
Name: AreaID, dtype: int64

In [166]:
premise_desc = crime_df.PremiseCode.value_counts() 
premise_desc.head(10)

101.0    38064
502.0    24341
501.0    22651
108.0    12778
102.0    11263
203.0     9227
122.0     6446
707.0     3320
210.0     3165
123.0     2574
Name: PremiseCode, dtype: int64

In [167]:
crime_code = crime_df.CrimeCode.value_counts() 
crime_code.head(10)

624    16820
330    14673
440    13046
510    11412
310    10286
740    10077
626     8903
230     8644
210     7344
420     6942
Name: CrimeCode, dtype: int64

In [168]:
report_district = crime_df.ReportingDistrict.value_counts() 
report_district.head(10)

645    2735
646    2247
363    1898
162    1857
636    1672
152    1502
666    1497
156    1327
111    1316
182    1184
Name: ReportingDistrict, dtype: int64

In [169]:
crime_df.dtypes

AreaID               object
ReportingDistrict    object
CrimeCode            object
VictimAge            object
PremiseCode          object
dtype: object

In [170]:
#COnvert the datatype of each column to numeric for machine learning
crime_df['AreaID'] = crime_df['AreaID'].apply(pd.to_numeric, errors='coerce')
crime_df['CrimeCode'] = crime_df['CrimeCode'].apply(pd.to_numeric, errors='coerce')
crime_df['ReportingDistrict'] = crime_df['ReportingDistrict'].apply(pd.to_numeric, errors='coerce')
crime_df['VictimAge'] = crime_df['VictimAge'].apply(pd.to_numeric, errors='coerce')
crime_df['PremiseCode'] = crime_df['PremiseCode'].apply(pd.to_numeric, errors='coerce')

crime_df = crime_df.fillna(0)
crime_df.reset_index()
crime_df.dtypes

AreaID                 int64
ReportingDistrict      int64
CrimeCode              int64
VictimAge            float64
PremiseCode          float64
dtype: object

In [211]:
# model_small_df = crime_df.loc[crime_df['CrimeCode'].isin(['646'])]
# 440 - 0.628928407174613
#3     26280
#6     20940
#7     18542
#2     18192
#9     17306
#5     16300
#4     15293
#8     15290
#13      816
model_small_df = crime_df.loc[crime_df['PremiseCode'].isin(['123'])]
model_small_df.head()

,AreaID,ReportingDistrict,CrimeCode,VictimAge,PremiseCode
36,7,736,510,16.0,123.0
136,1,119,480,26.0,123.0
149,1,138,420,45.0,123.0
274,1,152,740,71.0,123.0
411,1,182,330,79.0,123.0


In [212]:
model_small_df.count()

AreaID               2574
ReportingDistrict    2574
CrimeCode            2574
VictimAge            2574
PremiseCode          2574
dtype: int64

In [213]:
X = model_small_df.drop("CrimeCode", axis=1)
y = model_small_df["CrimeCode"]
feature_names = model_small_df.columns
print(X.shape, y.shape)

(2574, 4) (2574,)


In [214]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)
clf.score(X, y)

0.8574203574203574

In [215]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=20)
rf = rf.fit(X, y)
rf.score(X, y)

0.8523698523698524